# Debiasing Non-Gender Specific Words for Gender Equality.
This Ipython Notebook contains code to remove bias from Word Vectors of Non-Gender Specific Words like computer, engineer, fashion, doctor etc.<br>
Gender bias is not a healthy practice, neither in real life and should not be there in Natural Language Processing Systems.<br>
Ideas here are implemented from a paper by Boliukbasi et al. (https://arxiv.org/abs/1607.06520).<br>
The GloVe word embeddings are taken from Jeffrey Pennington, Richard Socher, and Christopher D. Manning. (https://nlp.stanford.edu/projects/glove/).

In [32]:
# Importing important Libraries.
import numpy as np

In [34]:
# This function returns words and python map/dictionary containing word embeddings.
def glove_vecs(glove_file):
    """Function Parameters: Path to glove vector.txt file"""
    # Open Text file in read mode 
    with open(glove_file, 'r') as f:
        # Creating empty set of words
        words = set()
        # Creating empty dictionary to store word:vector pair
        word_to_vec_map = {}
        
        # Reading file line by line
        # Each line is a string with space in between.
        for line in f:
            # Strip() function strips the string from both starting and end.
            # Split() function splits the string in separate elements around a space
            line = line.strip().split()
            # First element is the word 
            curr_word = line[0]
            # Adding the word to the set words.
            words.add(curr_word)
            # Adding current word and it's vector in the dictionary.
            word_to_vec_map[curr_word] = np.array(line[1:], dtype=np.float64)
            
    return words, word_to_vec_map

In [35]:
words, word_to_vec_map = glove_vecs('data/glove.6B.50d.txt')

In [36]:
# Finding cosine similarity between 2 vectors.
def cosine_similarity(a , b ):
    """Function Parameters: a , b are 2 different vectors whose cosine similarity is to be found."""
    
    cos_similarity = np.dot(a , b) / (np.linalg.norm(a) * np.linalg.norm(b))
        
    return cos_similarity    

In [37]:
# Finding distance between 2 vectors using L-2 Norm.
def L2distance(a , b):
    """Function Parameters: a , b are 2 different vectors whose cosine similarity is to be found."""
    
    l2_distance = np.linalg.norm(a - b)
    
    return l2_distance


# We can use any of the above 2 methods to find similarity between 2 vectors.

# Finding Word Analogy:
The function below finds the best possible word to complete an analogy, given 3 words.<br>
eg: man:woman :: boy: ?. Find the best word from the dictionary that can fit in place of '?'. 

In [38]:
def word_analogy(a, b, c, word_to_vec_map):
    """Function Parameters: a , b , c are 3 words
       words_to_vec_map: dictionary of word vectors."""
    
    # Converting a,b and c to lower case.
    a = a.lower()
    b = b.lower()
    c = c.lower()
    
    # Finding the vectors for the given words:
    
    a_vec = word_to_vec_map[a]
    b_vec = word_to_vec_map[b]
    c_vec = word_to_vec_map[c]
    
    # Getting all the words from the dictionary.
    
    words = word_to_vec_map.keys()
    
    # Setting maximum cosine similarity to large negative number.
    
    maximum_cosine_similarity = -500
    
    # Looping over all the words to find the best fit for the analogy.
    
    for w in words:
        
        # Skip a , b and c
        if w in [a,b,c]:
            continue
            
        cos_similarity = cosine_similarity(b_vec - a_vec , word_to_vec_map[w] - c_vec ) 
        
        if cos_similarity > maximum_cosine_similarity:
            # Overiting maximum_cosine_similarity 
            maximum_cosine_similarity = cos_similarity
            # Saving the best word giving maximum_cosine_similarity
            best_word = w
    
    
    return best_word
    

In [39]:
word_analogy('italy', 'italian', 'america', word_to_vec_map)

'american'

# Debiasing Non-Gender Specific Word Vectors.
How a word vector can be biased? Let us see the following example.

In [40]:
x = word_to_vec_map['woman'] - word_to_vec_map['man']

In [41]:
# Let us see similarity between some gender specific names and the vector'x'
names = ['ronaldo' , 'jack', 'marie' , 'priya']
for name in names:
    print(cosine_similarity(word_to_vec_map[name], x))

print("We see that male names have negative similarity and female names have positive similarity. That's OK because\nthe vector x is woman - man")    
    

-0.312447968503
-0.165662998616
0.315597935396
0.17632041839
We see that male names have negative similarity and female names have positive similarity. That's OK because
the vector x is woman - man


In [42]:
# Let us see similarity between some words that should be non-gender specific.
common_words = ['technology' , 'engineer' , 'doctor','grandfather','grandmother','literature']
for word in common_words:
    print(cosine_similarity(word_to_vec_map[word], x))

print("We see that words like technology , engineer are inclined towards man while literature is inclined towards woman.")    

-0.131937324476
-0.0803928049452
0.118952894109
0.0236297984509
0.384601436374
0.0647250443346
We see that words like technology , engineer are inclined towards man while literature is inclined towards woman.


# Neutralizing Non-Gender Specific Words.
To neutralize non-gender specific words, we need to have 0 similarity between the word from both man and woman. In other words, cosine similarity between the word and 'x' obtained above should be nearly 0.<br>
For this, we need to find the bias direction from the vector of the word and then subtract this from original word vector. 
# Bias Direction:
Bias direction is given by projection of the word vector 'w' onto the direction of vector x.
bias_direction = w . x/|x|

In [43]:
def neutralize(word , x , word_to_vec_map):
    # Extracting word vector from the dictionary.
    w = word_to_vec_map[word]
    
    # Finding the bias direction
    
    bias_direction = np.dot(w,x) * x /np.square((np.linalg.norm(x)))
    
    w_unbiased = w - bias_direction
    
    return w_unbiased

In [44]:
w = "literature"
print("cosine similarity between " + w + " and x, before neutralizing: ", cosine_similarity(word_to_vec_map["literature"], x))

e_unbiased = neutralize("literature", x, word_to_vec_map)
print("cosine similarity between " + w + " and x, after neutralizing: ", cosine_similarity(e_unbiased, x))

cosine similarity between literature and x, before neutralizing:  0.0647250443346
cosine similarity between literature and x, after neutralizing:  -3.12308857172e-17


# Equalizing Word Pairs:
We want word pairs like (father, mother) , (actor,actress) etc. to be equidistant from the words we neutralized above. Or in other words, these words should be equidistant from the non-bias axis/direction.<br>
In the function below, we apply the equalizing algorithms given by Boliukbasi et al. (https://arxiv.org/abs/1607.06520)

In [45]:
def equalize(w1, w2, bias_axis, word_to_vec_map):
    # Extracting vectors from dictionary.
    
    w1_vec = word_to_vec_map[w1]
    
    w2_vec = word_to_vec_map[w2]
    
    # The equations implemented below are described in the paper in the given link.
    mu = (w1_vec + w2_vec) / 2
    
    # Projection of mu over bias_axis and the orthogonal axis.
    mu_B = np.dot(mu,bias_axis) * bias_axis / np.square(np.linalg.norm(bias_axis))
    mu_orth = mu - mu_B
    
    w1_vecB = np.dot(w1_vec,bias_axis) * bias_axis / np.square(np.linalg.norm(bias_axis))
    w2_vecB = np.dot(w2_vec,bias_axis) * bias_axis / np.square(np.linalg.norm(bias_axis))
    
    w1_vecB_corrected = np.sqrt(np.absolute(1 - np.square(np.linalg.norm(mu_orth)))) * (w1_vecB - mu_B) / np.absolute((w1_vec - mu_orth) - mu_B) 
    w2_vecB_corrected = np.sqrt(np.absolute(1 - np.square(np.linalg.norm(mu_orth)))) * (w2_vecB - mu_B) / np.absolute((w2_vec - mu_orth) - mu_B)
    
    e1 = w1_vecB_corrected  + mu_orth
    e2 = w2_vecB_corrected  + mu_orth
    
    return e1 , e2

In [30]:
print("cosine similarities before equalizing:")
print("cosine_similarity(word_to_vec_map[\"man\"], gender) = ", cosine_similarity(word_to_vec_map["man"], x))
print("cosine_similarity(word_to_vec_map[\"woman\"], gender) = ", cosine_similarity(word_to_vec_map["woman"], x))
print()
e1, e2 = equalize("man", "woman", x, word_to_vec_map)
print("cosine similarities after equalizing:")
print("cosine_similarity(e1, gender) = ", cosine_similarity(e1, x))
print("cosine_similarity(e2, gender) = ", cosine_similarity(e2, x))

cosine similarities before equalizing:
cosine_similarity(word_to_vec_map["man"], gender) =  -0.117110957653
cosine_similarity(word_to_vec_map["woman"], gender) =  0.356666188463

cosine similarities after equalizing:
cosine_similarity(e1, gender) =  -0.716572752584
cosine_similarity(e2, gender) =  0.739659647493
